In [182]:
vars = [var for var in globals() if var[0] != '_']
for var in vars:
    del globals()[var]

del var, vars

In [183]:
import numpy as np
import torch
import scipy.optimize as sci_opt
import sys
sys.path.append('../')
import module_opt
import importlib
importlib.reload(module_opt)
from module_opt import *
import matplotlib.pyplot as plt


In [184]:
start_x = 0.0
start_y = 0.0
end_x = 10.0
end_y = 10.0
N = 45

x0 = np.linspace(start_x, end_x, N)
y0 = np.linspace(start_y, end_y, N)
dv0 = np.concatenate((x0, y0))

In [185]:
len(dv0)

90

In [186]:
int(dv0.shape[0]*.5)

45

In [187]:
x = dv0[:N]
y = dv0[N:]
1/((x - 5)**2 + (y - 5)**2 + 1)

array([0.01960784, 0.02147865, 0.0236282 , 0.02611417, 0.02900983,
       0.03240927, 0.03643481, 0.04124766, 0.0470634 , 0.05417506,
       0.06298803, 0.07407407, 0.08825675, 0.10674901, 0.13137894,
       0.16496251, 0.21190893, 0.27912341, 0.37694704, 0.51820128,
       0.70760234, 0.90636704, 1.        , 0.90636704, 0.70760234,
       0.51820128, 0.37694704, 0.27912341, 0.21190893, 0.16496251,
       0.13137894, 0.10674901, 0.08825675, 0.07407407, 0.06298803,
       0.05417506, 0.0470634 , 0.04124766, 0.03643481, 0.03240927,
       0.02900983, 0.02611417, 0.0236282 , 0.02147865, 0.01960784])

In [188]:
sum(1/((x - 5)**2 + (y - 5)**2 + 1))

np.float64(8.919456477298413)

In [189]:
np.arange(N-1).sum()

np.int64(946)

In [190]:
np.array([(x0[i+1] - x0[i])**2 + (y0[i+1] - y0[i])**2 for i in np.arange(N-1)]).sum()

np.float64(4.545454545454547)

In [191]:
N = 45 # The number of positions of drone
T = 15.0 # Total flight time [sec]
vmax = 1.5 # Maximum velocity [unit / sec]

def f(dv):
    N = int(dv.shape[0]*.5)
    x = dv[:N]
    y = dv[N:]
    C = (1/((x - 5)**2 + (y - 5)**2 + 1)).sum()
    d_sq = np.array([(x[i+1] - x[i])**2 + (y[i+1] - y[i])**2 for i in np.arange(N-1)]).sum()
    return C + d_sq

def ce1(dv): # x1 = 0.0
    return dv[0]

def ce2(dv): # y1 = 0.0
    N = int(dv.shape[0]*.5)
    return dv[N]

def ce3(dv): # xN = 10.0
    N = int(dv.shape[0]*.5)
    return dv[N-1] - 10.0

def ce4(dv): # yN = 10.0
    return dv[-1] - 10.0

def ce5(dv): # (x2 - x1)/(15/(N - 1)) = 0.0 <-> u1 = 0.0
    N = int(dv.shape[0]*.5)
    return (dv[1] - dv[0])/(T/(N - 1))

def ce6(dv): # (y2 - y1)/(15/(N - 1)) = 0.0 <-> v1 = 0.0
    N = int(dv.shape[0]*.5)
    return (dv[N+1] - dv[N])/(T/(N - 1))

ci_x_larger_0 = [lambda dv, i=i : dv[i] for i in np.arange(N)] # xi >= 0.0
ci_x_smaller_10 = [lambda dv, i=i : -dv[i] + 10.0 for i in np.arange(N)] # xi <= 10.0
ci_y_larger_0 = [lambda dv, i=i : dv[N+i] for i in np.arange(N)] # yi >= 0.0
ci_y_smaller_10 = [lambda dv, i=i : -dv[N+i] + 10.0 for i in np.arange(N)] # yi <= 10.0
ci_v_smaller_vmax = [lambda dv, i=i : vmax**2 - (((dv[i+1] - dv[i])/(T/(N - 1)))**2 + ((dv[N+i+1] - dv[N+i])/(T/(N - 1)))**2) for i in np.arange(N-1)] # vi^2 <= vmax^2

In [192]:
start_x = 0.0
start_y = 0.0
end_x = 10.0
end_y = 10.0
N = 45

x0 = np.linspace(start_x, end_x, N)
y0 = np.linspace(start_y, end_y, N)
dv0 = np.concatenate((x0, y0))



In [193]:
print(f(dv0))
print(ce1(dv0))
print(ce2(dv0))
print(ce3(dv0))
print(ce4(dv0))
print(ce5(dv0))
print(ce6(dv0))

13.46491102275296
0.0
0.0
0.0
0.0
0.6666666666666667
0.6666666666666667


In [194]:
for ci in ci_x_larger_0:
# for ci in ci_x_smaller_10:
# for ci in ci_y_larger_0:
# for ci in ci_y_smaller_10:
# for ci in ci_v_smaller_vmax:
    print(ci(dv0))

0.0
0.22727272727272727
0.45454545454545453
0.6818181818181818
0.9090909090909091
1.1363636363636362
1.3636363636363635
1.5909090909090908
1.8181818181818181
2.0454545454545454
2.2727272727272725
2.5
2.727272727272727
2.9545454545454546
3.1818181818181817
3.409090909090909
3.6363636363636362
3.8636363636363633
4.090909090909091
4.318181818181818
4.545454545454545
4.7727272727272725
5.0
5.2272727272727275
5.454545454545454
5.681818181818182
5.909090909090909
6.136363636363636
6.363636363636363
6.590909090909091
6.818181818181818
7.045454545454545
7.2727272727272725
7.5
7.727272727272727
7.954545454545454
8.181818181818182
8.409090909090908
8.636363636363637
8.863636363636363
9.09090909090909
9.318181818181818
9.545454545454545
9.772727272727272
10.0


In [195]:
ci = ci_x_larger_0 + ci_x_smaller_10 + ci_y_larger_0 + ci_y_smaller_10 + ci_v_smaller_vmax
alm(f=f, ce=[ce1, ce2, ce3, ce4, ce5, ce6], ci=ci, x0=dv0, inner_opt=3, tol=1e-3)

mu_0 = 1
rho_0 = 1
tau_0 = 0.01


KeyboardInterrupt: 